# Analysis and visualization of 3D data in Python

Daniela Ushizima, Alexandre de Siqueira, Stéfan van der Walt

_BIDS @ University of California, Berkeley_

_Lawrence Berkeley National Laboratory - LBNL_

* Support material for the tutorial _Analysis and visualization of 3D data in Python_.

This tutorial will introduce how to analyze three dimensional stacked and volumetric
images in Python, mainly using scikit-image. Here we will learn how to:
  * pre-process data using filtering, binarization and segmentation techniques.
  * inspect, count and measure attributes of objects and regions of interest in the data.
  * visualize 3D data.

Please prepare for the tutorial by [installing the pre-requisite
software](preparation.md) beforehand.

For more info:
  * [[ISVC 2019]](https://www.isvc.net/)
  * [[scikit-image]](https://scikit-image.org/)

## Using joblib to parallelize code

In [7]:
import numpy as np
from skimage import restoration, data, io

In [8]:
input_filename="data/cells.tif"


cells_rescaled = io.imread(input_filename)

In [9]:
def bilateral_classic_loop():
    cells_bilateral = np.empty_like(cells_rescaled)
    for plane, image in enumerate(cells_rescaled):
        cells_bilateral[plane] = restoration.denoise_bilateral(image, multichannel=False)
    return cells_bilateral

%timeit bilateral_classic_loop()

2.18 s ± 59.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
from joblib import Parallel, delayed

# when using n_jobs=-2, all CPUs but one are used.

def bilateral_joblib_loop():
    cells_bilateral = Parallel(n_jobs=-2)(delayed(restoration.denoise_bilateral)(plane,multichannel=False) for plane in cells_rescaled)

%timeit bilateral_joblib_loop()

815 ms ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Using numba to "compile" code

Numba is a just-in-time (or JIT) compiler, which translates a part of Python and Numpy into faster code.

The most basic way to use Numba is through the `@jit` *decorator*:

In [12]:
!conda install numba --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/dani/anaconda3/envs/imagexd19

  added / updated specs:
    - numba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    llvmlite-0.29.0            |   py37h98b8051_0        13.7 MB
    numba-0.45.1               |   py37h6440ff4_0         2.9 MB
    tbb-2019.4                 |       h04f5b5a_0         163 KB
    ------------------------------------------------------------
                                           Total:        16.8 MB

The following NEW packages will be INSTALLED:

  llvmlite           pkgs/main/osx-64::llvmlite-0.29.0-py37h98b8051_0
  numba              pkgs/main/osx-64::numba-0.45.1-py37h6440ff4_0
  tbb          

In [14]:
from numba import jit

This way, Numba decides what it should optimize in your functions. To show how fast this **decorator** is, let's define two versions of a dummy Fibonacci function. One `fibonacci_no_numba`, does not use Numba:

In [15]:
def fibonacci_no_numba(elem=10):
    """
    """
    fibonacci = np.zeros(elem)
    aux_1, aux_2 = 1, 1
    fibonacci[0: 2] = aux_1, aux_2

    for idx in range(2, elem):
        fibo_current = aux_1 + aux_2
        fibonacci[idx] = fibo_current
        aux_1 = aux_2
        aux_2 = fibo_current
    return fibonacci

The other, `fibonacci_numba`, uses Numba to optimize the function.

In [16]:
fibonacci_numba = jit()(fibonacci_no_numba)

Check that the functions are equal, and produce the same output:

In [17]:
elem = 9
print(f'* The {elem} first elements of the Fibonacci sequence, using fibonacci_no_numba: {fibonacci_no_numba(elem=elem)}')
print(f'* The {elem} first elements of the Fibonacci sequence, using fibonacci_numba: {fibonacci_numba(elem=elem)}')

* The 9 first elements of the Fibonacci sequence, using fibonacci_no_numba: [ 1.  1.  2.  3.  5.  8. 13. 21. 34.]
* The 9 first elements of the Fibonacci sequence, using fibonacci_numba: [ 1.  1.  2.  3.  5.  8. 13. 21. 34.]


Now we use the Jupyter Notebook magic `%timeit` to calculate how fast these functions return a Fibonacci sequence with 1000 elements:

In [20]:
time_no_numba = %timeit -o fibonacci_no_numba(elem=10000)

OverflowError: int too large to convert to float

In [21]:
time_numba = %timeit -o fibonacci_numba(elem=10000)

8.34 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
print(f'Numba version is around {int(time_no_numba.best / time_numba.best)} times faster than non-Numba one.')

Now, let's use its speed to ease our operations with images.